In [1]:
import requests
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd
import numpy as np

In [2]:
!pip install xlrd

     |████████████████████████████████| 112kB 22.8MB/s eta 0:00:01


In [49]:
df = pd.read_excel('6.xlsx')

In [4]:
!conda install -c conda-forge folium=0.10.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - folium=0.10.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge

In [5]:
import folium

In [6]:
latitude = 35.6804
longitude = 139.7690
print('The geograpical coordinate Chuo are {}, {}.'.format(latitude, longitude))

The geograpical coordinate Chuo are 35.6804, 139.769.


In [7]:
df

,Ward,District,Latitude,Longitude
0,Shinjuku,Agebacho,35.7024,139.7430
1,Shinjuku,Aizumicho,35.6902,139.7199
2,Shinjuku,Akagi Motomachi,35.7056,139.7393
3,Shinjuku,Akagi Shitamachi,35.7057,139.7343
4,Shinjuku,Arakicho,35.6902,139.7229
...,...,...,...,...
181,Minato,Shibadaimon,35.6542,139.7534
182,Minato,Shibakoen,35.6578,139.7476
183,Minato,Shirokane,35.6430,139.7269
184,Minato,Shirokanedai,35.6339,139.7297


In [50]:
map_6_ward = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, ward, district in zip(df['Latitude'], df['Longitude'], df['Ward'], df['District']):
    label = '{}, {}'.format(district, ward)
#     print(label)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_6_ward)
    
map_6_ward

In [38]:
CLIENT_ID = 'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

# CLIENT_ID = 'O1J3DJUGPSRN5DNX5DGXKYVUYEX3WBIPYKJ2ET3CEABC5QE5' # your Foursquare ID
# CLIENT_SECRET = 'HMBKOXTYYG41Z5KLRGKSMEKCMFDYBFLKZQ2WSGGKSPM4AD4J' # your Foursquare Secret

CLIENT_ID = 'FBRTSN1YMP0HEYRJOMOMC0P3TTOZPME50XDMIEQQRH5PA2IM'
CLIENT_SECRET = 'ODX200QGVCLGYHQCSBQ25MEVDXOK2J0W53UL4IKTDLGSRGP5'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FBRTSN1YMP0HEYRJOMOMC0P3TTOZPME50XDMIEQQRH5PA2IM
CLIENT_SECRET:ODX200QGVCLGYHQCSBQ25MEVDXOK2J0W53UL4IKTDLGSRGP5


In [39]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
df_1 = pd.read_excel('6_1.xlsx')
tokyo_venues_1 = getNearbyVenues(names=df_1['District'],
                               latitudes=df_1['Latitude'],
                               longitudes=df_1['Longitude']
                              )
df_2 = pd.read_excel('6_2.xlsx')
tokyo_venues_2 = getNearbyVenues(names=df_2['District'],
                               latitudes=df_2['Latitude'],
                               longitudes=df_2['Longitude']
                              )
df_3 = pd.read_excel('6_3.xlsx')
tokyo_venues_3= getNearbyVenues(names=df_3['District'],
                               latitudes=df_3['Latitude'],
                               longitudes=df_3['Longitude']
                              )

In [52]:
tokyo_venues = pd.concat([tokyo_venues_1,tokyo_venues_2,tokyo_venues_3],axis=0)

In [53]:
tokyo_venues.shape

(13315, 7)

In [54]:
tokyo_venues.head(15)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agebacho,35.7024,139.743,Kinozen (紀の善),35.700595,139.742369,Wagashi Place
1,Agebacho,35.7024,139.743,三州屋 飯田橋,35.703015,139.744219,Sake Bar
2,Agebacho,35.7024,139.743,Kagurazaka Uokin (神楽坂魚金),35.702105,139.741621,Seafood Restaurant
3,Agebacho,35.7024,139.743,Grilled Aging Beef (グリルド エイジング・ビーフ),35.701735,139.742386,Steakhouse
4,Agebacho,35.7024,139.743,BEER BAR Bitter,35.702833,139.740863,Bar
5,Agebacho,35.7024,139.743,El Pulpo,35.701791,139.740225,Tapas Restaurant
6,Agebacho,35.7024,139.743,Kohaku (虎白),35.700713,139.741101,Kaiseki Restaurant
7,Agebacho,35.7024,139.743,SAPANA,35.701038,139.744959,Indian Restaurant
8,Agebacho,35.7024,139.743,ギンレイホール,35.701414,139.743143,Movie Theater
9,Agebacho,35.7024,139.743,Cafe-Creperie Le Bretagne,35.701849,139.739727,French Restaurant
